# **Deblurring constrained DRED**

---

This code is mainly based on DeepRED code available at https://github.com/GaryMataev/DeepRED

This notebook is the implementation of the following paper: 

**Constrained and unconstrained deep image prior optimization models with automatic regularization** by *Pasquale Cascarano, Giorgia Franchini, Erich Kobler, Federica Porta and Andrea Sebastiani*

# Import libs

In [ ]:
import os
from threading import Thread 
import queue

import numpy as np
import torch
import torch.optim
from models.skip import skip  

from utils.utils import *  
from utils.mine_blur_utils2 import *  
from utils.data import Data  

from skimage.restoration import denoise_nl_means

from scipy.signal import convolve2d

In [ ]:
# got GPU? - if you are not getting the exact article results set CUDNN to False
CUDA_FLAG = True
CUDNN = True 
if CUDA_FLAG:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    # GPU accelerated functionality for common operations in deep neural nets
    torch.backends.cudnn.enabled = CUDNN
    # benchmark mode is good whenever your input sizes for your network do not vary.
    # This way, cudnn will look for the optimal set of algorithms for that particular 
    # configuration (which takes some time). This usually leads to faster runtime.
    # But if your input sizes changes at each iteration, then cudnn will benchmark every
    # time a new size appears, possibly leading to worse runtime performances.
    torch.backends.cudnn.benchmark = CUDNN
    # torch.backends.cudnn.deterministic = True
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor

# Constants

In [ ]:
NOISE_SIGMA = 5
STD_BLUR    = 1.6
DIM_FILTER  = 21
BLUR_TYPE = 'gauss_blur'  # 'gauss_blur' or 'uniform_blur' that the two only options
GRAY_SCALE = False        # if gray scale is False means we have rgb image, the psnr will be compared on Y. ch.
                          # if gray scale is True it will turn rgb to gray scale
USE_FOURIER = False

# graphs labels:
X_LABELS = ['Iterations']*3
Y_LABELS = ['PSNR between x and net (db)', 'PSNR with original image (db)', 'loss']

# Algorithm NAMES (to get the relevant image: use data_dict[alg_name].img)
# for example use data_dict['Clean'].img to get the clean image
ORIGINAL  = 'Clean'
CORRUPTED = 'Blurred'
DIP_NLM   = 'cDIP-RED (NLM)'

# Load image for DeBlurring

In [ ]:
def load_imgs_deblurring(fname, blur_type, noise_sigma,STD_BLUR, DIM_FILTER,plot=False):
    """  Loads an image, and add gaussian blur
    Args: 
         fname: path to the image
         blur_type: 'uniform' or 'gauss'
         noise_sigma: noise added after blur
         covert2gray: should we convert to gray scale image?
         plot: will plot the images
    Out:
         dictionary of images and dictionary of psnrs
    """
    img_pil, img_np = load_and_crop_image(fname)        
    if GRAY_SCALE:
        img_np = rgb2gray(img_pil)
    kernel = get_h(blur_type,STD_BLUR,DIM_FILTER)
    kernel_torch = np_to_torch(kernel)  
    blurred = torch_to_np(blur_th(np_to_torch(img_np), kernel_torch))
    blurred = np.clip(blurred + np.random.normal(scale=noise_sigma/255., size=blurred.shape), 0, 1).astype(np.float32)
    data_dict = { ORIGINAL: Data(img_np), 
                 CORRUPTED: Data(blurred, compare_PSNR(img_np, blurred,   on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE)) }
    if plot:
        plot_dict(data_dict)
    return data_dict,kernel_torch

In [ ]:
# Get the LR and HR images
data_dict,kernel_torch = load_imgs_deblurring('datasets/watercastle.png', BLUR_TYPE, NOISE_SIGMA,STD_BLUR, DIM_FILTER, plot=True)

#  ESTIMATING THE NOISE

In [ ]:
lap_kernel = np.array([[1,-2,1], [-2, 4, -2], [1,-2,1]])
h=data_dict[CORRUPTED].img[:,:,:].shape[2]
w=data_dict[CORRUPTED].img[:,:,:].shape[1]

def estimate_variance(img):
  out = convolve2d(img, lap_kernel, mode='valid')
  out = np.sum(np.abs(out))
  out = (out*np.sqrt(0.5*np.pi)/(6*(h-2)*(w-2)))
  return out

print(data_dict[CORRUPTED].img[:,:,:].shape)
NOISE_SIGMA = estimate_variance(data_dict[CORRUPTED].img[0,:,:])*255
print(NOISE_SIGMA)

# THE NETWORK

In [ ]:
def get_network_and_input(img_shape, input_depth=32, pad='reflection',
                          upsample_mode='bilinear', use_interpolate=True, align_corners=False,
                          act_fun='LeakyReLU', skip_n33d=128, skip_n33u=128, skip_n11=4,
                          num_scales=5, downsample_mode='stride', INPUT='noise'):  # 'meshgrid'
    """ Getting the relevant network and network input (based on the image shape and input depth)
        We are using the same default params as in DIP article
        img_shape - the image shape (ch, x, y)
    """
    n_channels = img_shape[0]
    net = skip(input_depth, n_channels,
               num_channels_down=[skip_n33d] * num_scales if isinstance(skip_n33d, int) else skip_n33d,
               num_channels_up=[skip_n33u] * num_scales if isinstance(skip_n33u, int) else skip_n33u,
               num_channels_skip=[skip_n11] * num_scales if isinstance(skip_n11, int) else skip_n11,
               upsample_mode=upsample_mode, use_interpolate=use_interpolate, align_corners=align_corners,
               downsample_mode=downsample_mode, need_sigmoid=True, need_bias=True, pad=pad, act_fun=act_fun).type(dtype)
    net_input = get_noise(input_depth, INPUT, img_shape[1:]).type(dtype).detach()
    return net, net_input

## The Non Local Means denoiser

In [ ]:
def non_local_means(noisy_np_img, sigma, fast_mode=True):
    """ get a numpy noisy image
        returns a denoised numpy image using Non-Local-Means
    """ 
    sigma = sigma / 255.
    h = 0.6 * sigma if fast_mode else 0.8 * sigma
    patch_kw = dict(h=h,                   # Cut-off distance, a higher h results in a smoother image
                    sigma=sigma,           # sigma provided
                    fast_mode=fast_mode,   # If True, a fast version is used. If False, the original version is used.
                    patch_size=5,          # 5x5 patches (Size of patches used for denoising.)
                    patch_distance=6,      # 13x13 search area
                    multichannel=False)
    denoised_img = []
    n_channels = noisy_np_img.shape[0]
    for c in range(n_channels):
        denoise_fast = denoise_nl_means(noisy_np_img[c, :, :], **patch_kw)
        denoised_img += [denoise_fast]
    return np.array(denoised_img, dtype=np.float32)

# Constrained Deep Image prior with RED

In [ ]:
def train_via_admm(net, net_input, denoiser_function,kernel_torch,y, noise_lev,tau, clean_img=None,                    
                   plot_array={}, algorithm_name="", gamma=.9, step_size=1000, save_path="", 
                   admm_iter=5000, LR=0.004,                                                                      
                   sigma_f=3, update_iter=10, method='fixed_point', 
                   beta=1, mu=0.5,mu_r=1, LR_x=None, noise_factor=0.033): 

    """ training the network using
        ## Must Params ##
        net                 - the network to be trained
        net_input           - the network input
        denoiser_function   - an external denoiser function, used as black box, this function
                              must get numpy noisy image, and return numpy denoised image
        y                   - the noisy image
        sigma               - the noise level (int 0-255)
        
        # optional params #
        org_img             - the original image if exist for psnr compare only, or None (default)
        plot_array          - prints params at the begging of the training and plot images at the required indices
        admm_iter           - total number of admm epoch
        LR                  - the lr of the network in admm (step 2)
        sigma_f             - the sigma to send the denoiser function
        update_iter         - denoised image updated every 'update_iter' iteration
        method              - 'fixed_point' or 'grad' or 'mixed' 
        algorithm_name      - the name that would show up while running, just to know what we are running ;)
                
        # equation params #  
        beta                - regularization parameter (lambda in the article)
        mu                  - ADMM parameter
        LR_x                - learning rate of the parameter x, needed only if method!=fixed point
        # more
        noise_factor       - the amount of noise added to the input of the network
        threshold          - when the image become close to the noisy image at this psnr
        increase_reg       - we going to increase regularization by this amount
        threshold_step     - and keep increasing it every step
    """
    
    # get optimizer and loss function:
    mse = torch.nn.MSELoss().type(dtype)  # using MSE loss
    
    # additional noise added to the input:
    net_input_saved = net_input.detach().clone()
    noise = net_input.detach().clone()

    if clean_img is not None:
        psnr_y = compare_PSNR(clean_img, y,  on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE)
    
    # x update method:
    if method == 'fixed_point':
        swap_iter = admm_iter + 1
        LR_x = None
    elif method == 'grad':
        swap_iter = -1
    elif method == 'mixed':
        swap_iter = admm_iter // 2
    else:
        assert False, "method can be 'fixed_point' or 'grad' or 'mixed' only "
    
    # optimizer and scheduler
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)  # using ADAM opt
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=gamma, step_size=step_size)
    
    y_torch = np_to_torch(y).type(dtype)

    x, u,u_r, r, out_np, res,out_np_old = y.copy(), np.zeros_like(y),np.zeros_like(y),y.copy(), np.zeros_like(y),y.copy(),np.zeros_like(y)
    f_x, avg, avg2, avg3 = x.copy(), np.rint(y), np.rint(y), np.rint(y)
    
    list_psnr=[]
    list_stopping=[]
    
    img_queue = queue.Queue()
    denoiser_thread = Thread(target=lambda q, f, f_args: q.put(f(*f_args)),
                             args=(img_queue, denoiser_function, [x.copy(), sigma_f]))
    denoiser_thread.start()

    for i in range(1, 1 + admm_iter):

        rho = tau*noise_lev*np.sqrt(y.shape[0]*y.shape[1]*y.shape[2] - 1)  

        # step 1, update x using a denoiser and result from step 1
        if i % update_iter == 0:  # the denoiser work in parallel
            denoiser_thread.join()
            f_x = img_queue.get()
            denoiser_thread = Thread(target=lambda q, f, f_args: q.put(f(*f_args)),
                                     args=(img_queue, denoiser_function, [x.copy(), sigma_f]))
            denoiser_thread.start()

        if i < swap_iter:
            x = 1 / (beta + mu) * (beta * f_x + mu * (out_np + u))
        else:
            x = x - LR_x * (beta * (x - f_x) + mu * (x - out_np - u))
        
        #np.clip(x, 0, 1, out=x)  # making sure that image is in bounds

        # step 2 projection
        r = res + u_r  
        r_norm = np.sqrt(np.sum(np.square(r)))
        
        if r_norm > rho:
          r = r*(rho/r_norm)
          r_norm_2 = np.sqrt(np.sum(np.square(r)))

        # step 3, update network:
        optimizer.zero_grad()
        net_input = net_input_saved + (noise.normal_() * noise_factor)
        out = net(net_input)
        out_np = torch_to_np(out.data)

        # loss:
        loss_y = mse(blur_th(out, kernel_torch) - y_torch, np_to_torch(r-u_r).type(dtype))   
        loss_x = mse(out, np_to_torch(x - u).type(dtype))

        total_loss = mu_r*loss_y + mu * loss_x
        total_loss.backward()
        optimizer.step()
        scheduler.step()


        res = torch_to_np(blur_th(out.data, kernel_torch))-y

        
        # step 4, update u and u_r
        u = u +  0.01*(out_np - x)
        u_r = u_r + 0.01*(res - r)
        
        # Averaging:
        avg = avg * .99 + out_np * .01

        stopping = np.sqrt(np.sum(np.square(torch_to_np(blur_th(out.data, kernel_torch))-y)))/ rho 
        list_stopping.append(stopping)
        
        out_np_old = out_np

        # show psnrs:
        psnr_noisy = compare_PSNR(out_np, y, on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE)
        
        if clean_img is not None:
            psnr_net, psnr_avg = compare_PSNR(clean_img, out_np, on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE), compare_PSNR(clean_img, avg, on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE)
            psnr_x, psnr_x_u = compare_PSNR(clean_img, x, on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE), compare_PSNR(clean_img, x - u, on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE)
            list_psnr.append(psnr_avg)
            print('\r', algorithm_name, '%04d/%04d Loss %f' % (i, admm_iter, total_loss.item()),
                  'psnrs: y: %.2f psnr_noisy: %.2f net: %.2f' % (psnr_y, psnr_noisy, psnr_net),
                  'x: %.2f x-u: %.2f avg: %.2f' % (psnr_x, psnr_x_u, psnr_avg), 
                  'params: rho: %.2f r_norm: %.2f  r_norm2: %.2f stopping: %.2f mu: %.2f' % (rho,r_norm,r_norm_2,stopping,mu), end='')
            if i in plot_array:  # plot images
                tmp_dict = {'Clean': Data(clean_img),
                            'Blurred': Data(y, psnr_y),
                            'Net': Data(out_np, psnr_net),
                            #'x-u': Data(x - u, psnr_x_u),
                            'avg': Data(avg, psnr_avg),
                            #'x': Data(x, psnr_x),
                            'u': Data((u - np.min(u)) / (np.max(u) - np.min(u))),
                            'u_r': Data((u_r - np.min(u_r)) / (np.max(u_r) - np.min(u_r))),
                            'r': Data(r)
                            }
                plot_dict(tmp_dict)
        else:
            print('\r', algorithm_name, 'iteration %04d/%04d Loss %f' % (i, admm_iter, total_loss.item()), end='')
    
    if denoiser_thread.is_alive():
        denoiser_thread.join()  # joining the thread
    return avg,list_psnr,list_stopping

## Let's Go:

In [ ]:
def run_and_plot(denoiser, name, plot_checkpoints={}):
    global data_dict
    noise_lev = NOISE_SIGMA/255
    tau=1 # keep it a 1 if you trust the noise estimated level 
    net, net_input = get_network_and_input(img_shape=data_dict[CORRUPTED].img.shape)
    denoised_img,list_psnr,list_stopping = train_via_admm(net, net_input, denoiser,kernel_torch, data_dict[CORRUPTED].img,noise_lev,tau,
                                  plot_array=plot_checkpoints, algorithm_name=name,
                                  clean_img=data_dict[ORIGINAL].img)
    data_dict[name] = Data(denoised_img, compare_PSNR(data_dict[ORIGINAL].img, denoised_img,on_y=(not GRAY_SCALE), gray_scale=GRAY_SCALE))
    plot_dict(data_dict)

    return denoised_img,list_psnr,list_stopping


plot_checkpoints = {1, 10, 50, 100, 250, 500, 800, 1000, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3500, 5000} 
denoised_img,list_psnr,list_stopping=run_and_plot(non_local_means, DIP_NLM, plot_checkpoints)  # you may try it with different denoisers
